In [1]:
import pandas as pd
import dbcp

/app/.local/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.
  warnings.warn(
/app/.local/lib/python3.10/site-packages/pudl/analysis/spatial.py:7: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/

In [2]:
engine = dbcp.helpers.get_sql_engine()
with engine.connect() as con:
    br_elections = pd.read_sql_table("br_elections", con, schema="data_warehouse")
    br_races = pd.read_sql_table("br_races", con, schema="data_warehouse")
    br_positions = pd.read_sql_table("br_positions", con, schema="data_warehouse")

In [3]:
commissioner_positions = br_positions.query("tier > 2 & is_judicial == False & normalized_position_id in (910,912)")

In [4]:
commissioner_positions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10192 entries, 168 to 37364
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   position_id               10192 non-null  int64 
 1   position_name             10192 non-null  object
 2   reference_year            10192 non-null  int64 
 3   sub_area_name             9095 non-null   object
 4   sub_area_value            9629 non-null   object
 5   sub_area_name_secondary   92 non-null     object
 6   sub_area_value_secondary  114 non-null    object
 7   level                     10192 non-null  object
 8   tier                      10192 non-null  int64 
 9   is_judicial               10192 non-null  bool  
 10  is_retention              10192 non-null  bool  
 11  normalized_position_id    10192 non-null  int64 
 12  normalized_position_name  10192 non-null  object
 13  frequency                 10192 non-null  object
 14  partisan_type       

In [5]:
position_fields = [
    "position_id",
    "position_name",
    "reference_year",
    "frequency"
]
commissioner_positions = commissioner_positions[position_fields]

In [7]:
engine = dbcp.helpers.get_sql_engine()
with engine.connect() as con:
    br_election_data = pd.read_sql_table("br_election_data", con, schema="data_mart")

In [8]:
commissioner_races = br_election_data.query("tier > 2 & is_judicial == False & normalized_position_id in (910,912)")

In [9]:
pos_grp = commissioner_races.groupby(["election_id", "county_id_fips"]).nunique().position_id

In [10]:
pos_grp

election_id  county_id_fips
4217         53001             2
             53003             2
             53005             2
             53007             2
             53009             1
                              ..
6909         40145             1
             40147             1
             40149             1
             40151             1
             40153             1
Name: position_id, Length: 6708, dtype: int64

In [11]:
pos_grp[pos_grp > 1].sort_values().tail(40)

election_id  county_id_fips
5194         22047             37
4718         22047             37
5194         22007             37
4706         55025             37
4718         22017             38
5292         55073             38
5194         22057             38
4718         22057             38
5194         22073             38
4706         55073             38
4718         22073             38
5194         22017             38
4407         36001             39
5194         22009             39
5225         36001             39
4718         22009             39
             22079             40
5194         22079             40
4718         22033             40
5194         22033             40
             22013             41
             22059             41
4718         22059             41
             22013             41
5194         22099             45
4718         22099             45
5194         22041             45
4718         22061             45
             22041  

In [12]:
commissioner_races.query("election_id == 5225 & county_id_fips == '36001'")

,state_name,county_name,raw_county,election_id,election_name,election_day,race_id,is_primary,is_runoff,is_unexpired,position_id,position_name,sub_area_name,sub_area_value,sub_area_name_secondary,sub_area_value_secondary,raw_state,level,tier,is_judicial,is_retention,number_of_seats,normalized_position_id,normalized_position_name,frequency,reference_year,partisan_type,race_created_at,race_updated_at,state_id_fips,county_id_fips
69298,New York,Albany,Albany County,5225,New York Primary Election,2023-06-27,1627002,True,False,False,263057,Albany County Legislature - District 19,District,19,None,None,NY,county,3,False,False,1,910,County Legislature//Executive Board,[4],2023,partisan,2020-01-14 23:19:50.894,2023-04-05 14:30:17.020,36,36001
69299,New York,Albany,Albany County,5225,New York Primary Election,2023-06-27,1627008,True,False,False,263068,Albany County Legislature - District 29,District,29,None,None,NY,county,3,False,False,1,910,County Legislature//Executive Board,[4],2023,partisan,2020-01-14 23:19:50.894,2023-04-05 14:30:18.781,36,36001
69300,New York,Albany,Albany County,5225,New York Primary Election,2023-06-27,1627040,True,False,False,263073,Albany County Legislature - District 33,District,33,None,None,NY,county,3,False,False,1,910,County Legislature//Executive Board,[4],2023,partisan,2020-01-14 23:19:50.894,2023-04-05 14:30:18.884,36,36001
69301,New York,Albany,Albany County,5225,New York Primary Election,2023-06-27,1627041,True,False,False,263062,Albany County Legislature - District 23,District,23,None,None,NY,county,3,False,False,1,910,County Legislature//Executive Board,[4],2023,partisan,2020-01-14 23:19:50.894,2023-04-05 14:30:17.854,36,36001
69303,New York,Albany,Albany County,5225,New York Primary Election,2023-06-27,1627061,True,False,False,263060,Albany County Legislature - District 21,District,21,None,None,NY,county,3,False,False,1,910,County Legislature//Executive Board,[4],2023,partisan,2020-01-14 23:19:50.894,2023-04-05 14:30:17.781,36,36001
69304,New York,Albany,Albany County,5225,New York Primary Election,2023-06-27,1627064,True,False,False,263081,Albany County Legislature - District 5,District,5,None,None,NY,county,3,False,False,1,910,County Legislature//Executive Board,[4],2023,partisan,2020-01-14 23:19:50.894,2023-04-05 14:30:19.044,36,36001
69305,New York,Albany,Albany County,5225,New York Primary Election,2023-06-27,1627066,True,False,False,263076,Albany County Legislature - District 36,District,36,None,None,NY,county,3,False,False,1,910,County Legislature//Executive Board,[4],2023,partisan,2020-01-14 23:19:50.894,2023-04-05 14:30:18.966,36,36001
69307,New York,Albany,Albany County,5225,New York Primary Election,2023-06-27,1627082,True,False,False,263077,Albany County Legislature - District 37,District,37,None,None,NY,county,3,False,False,1,910,County Legislature//Executive Board,[4],2023,partisan,2020-01-14 23:19:50.894,2023-04-05 14:30:18.875,36,36001
69308,New York,Albany,Albany County,5225,New York Primary Election,2023-06-27,1627091,True,False,False,263083,Albany County Legislature - District 7,District,7,None,None,NY,county,3,False,False,1,910,County Legislature//Executive Board,[4],2023,partisan,2020-01-14 23:19:50.894,2023-04-05 14:30:19.032,36,36001
69313,New York,Albany,Albany County,5225,New York Primary Election,2023-06-27,1627123,True,False,False,263051,Albany County Legislature - District 13,District,13,None,None,NY,county,3,False,False,1,910,County Legislature//Executive Board,[4],2023,partisan,2020-01-14 23:19:50.894,2023-04-05 14:30:16.890,36,36001


Looks like there are multiple districts per county which makes sense.

In [13]:
la_races = commissioner_races.query("election_id == 4718 & county_id_fips == '22069'")
la_races

,state_name,county_name,raw_county,election_id,election_name,election_day,race_id,is_primary,is_runoff,is_unexpired,position_id,position_name,sub_area_name,sub_area_value,sub_area_name_secondary,sub_area_value_secondary,raw_state,level,tier,is_judicial,is_retention,number_of_seats,normalized_position_id,normalized_position_name,frequency,reference_year,partisan_type,race_created_at,race_updated_at,state_id_fips,county_id_fips
16152,Louisiana,Natchitoches,Natchitoches Parish,4718,Louisiana General Election,2023-11-18,1454013,False,False,False,239163,Sabine Parish Police Juror - District 7,District,7,None,None,LA,county,3,False,False,1,910,County Legislature//Executive Board,[4],2023,partisan,2020-01-14 23:12:39.199,2021-09-30 19:37:57.406,22,22069
16274,Louisiana,Natchitoches,Natchitoches Parish,4718,Louisiana General Election,2023-11-18,1454034,False,False,False,239046,Grant Parish Police Juror - District 4,District,4,None,None,LA,county,3,False,False,1,910,County Legislature//Executive Board,[4],2023,partisan,2020-01-14 23:12:39.199,2021-09-30 19:37:57.406,22,22069
16316,Louisiana,Natchitoches,Natchitoches Parish,4718,Louisiana General Election,2023-11-18,1454051,False,False,False,239260,Vernon Parish Police Juror - District 8,District,8,None,None,LA,county,3,False,False,1,910,County Legislature//Executive Board,[4],2023,partisan,2020-01-14 23:12:39.199,2021-09-30 19:37:57.406,22,22069
16331,Louisiana,Natchitoches,Natchitoches Parish,4718,Louisiana General Election,2023-11-18,1454053,False,False,False,238931,Bienville Parish Police Juror - District 4,District,4,None,None,LA,county,3,False,False,1,910,County Legislature//Executive Board,[4],2023,partisan,2020-01-14 23:12:39.199,2021-09-30 19:37:57.406,22,22069
16361,Louisiana,Natchitoches,Natchitoches Parish,4718,Louisiana General Election,2023-11-18,1454060,False,False,False,238928,Bienville Parish Police Juror - District 1,District,1,None,None,LA,county,3,False,False,1,910,County Legislature//Executive Board,[4],2023,partisan,2020-01-14 23:12:39.199,2021-09-30 19:37:57.406,22,22069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19504,Louisiana,Natchitoches,Natchitoches Parish,4718,Louisiana General Election,2023-11-18,1454761,False,False,False,239294,Winn Parish Police Juror - District 6,District,6,None,None,LA,county,3,False,False,1,910,County Legislature//Executive Board,[4],2023,partisan,2020-01-14 23:12:39.199,2021-09-30 19:37:57.406,22,22069
19567,Louisiana,Natchitoches,Natchitoches Parish,4718,Louisiana General Election,2023-11-18,1454773,False,False,False,238933,Bienville Parish Police Juror - District 6,District,6,None,None,LA,county,3,False,False,1,910,County Legislature//Executive Board,[4],2023,partisan,2020-01-14 23:12:39.199,2021-09-30 19:37:57.406,22,22069
19611,Louisiana,Natchitoches,Natchitoches Parish,4718,Louisiana General Election,2023-11-18,1454783,False,False,False,239137,Rapides Parish Police Juror - District F,District,F,None,None,LA,county,3,False,False,1,910,County Legislature//Executive Board,[4],2023,partisan,2020-01-14 23:12:39.199,2021-09-30 19:37:57.406,22,22069
19687,Louisiana,Natchitoches,Natchitoches Parish,4718,Louisiana General Election,2023-11-18,1454804,False,False,False,239258,Vernon Parish Police Juror - District 6,District,6,None,None,LA,county,3,False,False,1,910,County Legislature//Executive Board,[4],2023,partisan,2020-01-14 23:12:39.199,2021-09-30 19:37:57.406,22,22069


In [14]:
la_races[la_races.position_name.str.contains("Natchitoches")]

,state_name,county_name,raw_county,election_id,election_name,election_day,race_id,is_primary,is_runoff,is_unexpired,position_id,position_name,sub_area_name,sub_area_value,sub_area_name_secondary,sub_area_value_secondary,raw_state,level,tier,is_judicial,is_retention,number_of_seats,normalized_position_id,normalized_position_name,frequency,reference_year,partisan_type,race_created_at,race_updated_at,state_id_fips,county_id_fips
17290,Louisiana,Natchitoches,Natchitoches Parish,4718,Louisiana General Election,2023-11-18,1454280,False,False,False,238799,Natchitoches Parish Council - District 3,District,3,None,None,LA,county,3,False,False,1,910,County Legislature//Executive Board,[4],2023,partisan,2020-01-14 23:12:39.199,2023-07-06 15:30:10.177,22,22069
17336,Louisiana,Natchitoches,Natchitoches Parish,4718,Louisiana General Election,2023-11-18,1454291,False,False,False,238801,Natchitoches Parish Council - District 5,District,5,None,None,LA,county,3,False,False,1,910,County Legislature//Executive Board,[4],2023,partisan,2020-01-14 23:12:39.199,2023-07-06 15:30:10.361,22,22069
18234,Louisiana,Natchitoches,Natchitoches Parish,4718,Louisiana General Election,2023-11-18,1454509,False,False,False,238797,Natchitoches Parish Council - District 1,District,1,None,None,LA,county,3,False,False,1,910,County Legislature//Executive Board,[4],2023,partisan,2020-01-14 23:12:39.199,2023-07-06 15:30:10.107,22,22069
18280,Louisiana,Natchitoches,Natchitoches Parish,4718,Louisiana General Election,2023-11-18,1454524,False,False,False,238800,Natchitoches Parish Council - District 4,District,4,None,None,LA,county,3,False,False,1,910,County Legislature//Executive Board,[4],2023,partisan,2020-01-14 23:12:39.199,2023-07-06 15:30:10.740,22,22069
18353,Louisiana,Natchitoches,Natchitoches Parish,4718,Louisiana General Election,2023-11-18,1454548,False,False,False,238798,Natchitoches Parish Council - District 2,District,2,None,None,LA,county,3,False,False,1,910,County Legislature//Executive Board,[4],2023,partisan,2020-01-14 23:12:39.199,2023-07-06 15:30:09.873,22,22069


It looks like LA has some straight up duplicates?

In [15]:
county_name_in_position = commissioner_races.apply(lambda row: row.county_name in row.position_name, axis=1)

In [16]:
county_name_in_position.value_counts()

True     23563
False     6005
dtype: int64

In [17]:
incorrect_counties = commissioner_races[~county_name_in_position]
incorrect_counties[["race_id", "county_name", "state_name", "raw_county", "position_name"]].head(10)

,race_id,county_name,state_name,raw_county,position_name
716,1371024,Benewah,Idaho,Benewah County,Latah County Commission - District 3
717,1371024,Clearwater,Idaho,Clearwater County,Latah County Commission - District 3
719,1371024,Nez Perce,Idaho,Nez Perce County,Latah County Commission - District 3
721,1371025,Camas,Idaho,Camas County,Blaine County Council - District 3
722,1371025,Power,Idaho,Power County,Blaine County Council - District 3
724,1371026,Minidoka,Idaho,Minidoka County,Cassia County Council - District 3
725,1371027,Caribou,Idaho,Caribou County,Franklin County Council - District 3
728,1371029,Idaho,Idaho,Idaho County,Lewis County Council - District 3
730,1371029,Nez Perce,Idaho,Nez Perce County,Lewis County Council - District 3
731,1371030,Canyon,Idaho,Canyon County,Payette County Council - District 3


- race_id = 1371024: Benewah, Clearwater, and Nez Perce are different counties than Latah county
- race_id (2778451 - 2778454): This one looks reasonable: nashville city council is in Davidson County.
- race_id = 1371025: camas and power are adjacent counties to Blaine.
- 

In [ ]:
incorrect_counties.position_name.value_counts()

In [ ]:
incorrect_counties.position_name.str.contains("-").value_counts()

In [ ]:
# Janky string clustering
# incorrect_counties.position_name.str.split("-").str[0].str.split(" ").str[1:].str.join(" ").value_counts()

In [ ]:
incorrect_counties.position_name.str.contains("County|Parish").value_counts()

## Clean incorrect comissions races

In [ ]:
county_name_in_position = commissioner_races.apply(lambda row: row.county_name in row.position_name, axis=1)

commissioner_races_with_name = commissioner_races[county_name_in_position].copy()
commissioner_races_without_name = commissioner_races[~county_name_in_position].copy()

In [ ]:
commissioner_races_without_name

In [ ]:
is_likely_incorrectly_geocoded_comissioner_races = commissioner_races_without_name.position_name.str.contains("County|Parish")
is_likely_incorrectly_geocoded_comissioner_races.value_counts()

In [ ]:
corrected_comissioner_races = commissioner_races_with_name

In [ ]:
corrected_comissioner_races[~corrected_comissioner_races.position_name.str.contains("County Commission")].sample(10)

- AS: Justice of the Peace
- LA: Police Juror
- WI: Board of Supervisors
- IL: County Board
- WA: County Council
- NY: County Legislature, County Legislature

In [ ]:
corrected_comissioner_races.position_name.str.contains("City Council").value_counts()

In [ ]:
corrected_comissioner_races = corrected_comissioner_races[~corrected_comissioner_races.position_name.str.contains("City Council")]

## relationships

In [ ]:
grouped_races = corrected_comissioner_races.groupby(["election_id", "county_id_fips"])
grouped_races.nunique().reference_year.value_counts()

In [ ]:
grouped_races.nunique().reference_year[grouped_races.nunique().reference_year > 1]

In [ ]:
fields = ["position_name", "election_name", "reference_year"]
corrected_comissioner_races.query("election_id == 4245 & county_id_fips == '16001'")[fields]

In [ ]:
mode = lambda x: x.value_counts().index[0]

grp_fields = ["election_id", "county_id_fips", "election_name", "election_day"]
agg_funcs = {
    "position_id": "count",
    "number_of_seats": "sum",
    "position_name": lambda x: ','.join(x),
    "frequency": mode,
    "reference_year": mode
}

rename_dict = {
    "number_of_seats": "total_n_of_seats",
    "position_id": "total_n_races",
    "position_name": "all_race_names",
}

comissioner_elections = corrected_comissioner_races.groupby(grp_fields).agg(agg_funcs).reset_index()
comissioner_elections = comissioner_elections.rename(columns=rename_dict)

assert ~comissioner_elections.duplicated(subset=["county_id_fips", "election_id"]).any(), "County comissioner election primary key is not unique."
assert (comissioner_elections.total_n_of_seats >= comissioner_elections.total_n_races).all()

In [ ]:
comissioner_elections

## Create wide

In [ ]:
from dbcp.data_mart.br_election_data import _create_br_election_data_mart, _create_county_commission_elections_long, _create_county_commission_elections_wide
import dbcp

engine = dbcp.helpers.get_sql_engine()

dfs = {}

dfs["br_election_data"] = _create_br_election_data_mart(engine)

county_commission_elections_long = _create_county_commission_elections_long(
        dfs["br_election_data"]
    )

county_commission_elections_wide = _create_county_commission_elections_wide(
        county_commission_elections_long
    )

In [ ]:
county_commission_elections_wide

In [ ]:
e_type = "primary"
invalid = county_commission_elections_wide[f"next_{e_type}_total_n_seats"] >= county_commission_elections_wide[f"next_{e_type}_total_n_races"]

invalid.all()

In [ ]:
with engine.connect() as con:
    df = pd.read_sql_table(
        "county_commission_election_info", con, schema="data_mart"
    ).convert_dtypes()
    

assert (
    county_commission_elections_wide.next_primary_total_n_seats >= county_commission_elections_wide.next_primary_total_n_races
).all(), "Found more races than seats in county_commission_election_info!"
assert (
    df.next_general_total_n_seats >= df.next_general_total_n_races
).all(), "Found more races than seats in county_commission_election_info!"
assert (
    df.next_run_off_total_n_seats >= df.next_run_off_total_n_races
).all(), "Found more races than seats in county_commission_election_info!"

In [ ]:
df.info()

In [ ]:
invalid = ~(df.next_primary_total_n_seats >= df.next_primary_total_n_races)


df[invalid]

In [ ]:
df.shape

In [ ]:
county_commission_elections_wide.shape

In [ ]:
import pandas as pd
county_commission_elections_long["election_type"] = pd.Series()

county_commission_elections_long["election_type"] = county_commission_elections_long.election_type.mask(county_commission_elections_long.is_primary, "primary")
county_commission_elections_long["election_type"] = county_commission_elections_long.election_type.mask(county_commission_elections_long.is_runoff, "run_off")
county_commission_elections_long["election_type"] = county_commission_elections_long["election_type"].fillna("general")

county_commission_elections_long = county_commission_elections_long.drop(columns=["is_primary", "is_runoff"])

In [ ]:
county_commission_elections_long["election_type"].value_counts()

In [ ]:
county_commission_elections_long

Can a county have multiple upcoming elections?

In [ ]:
for election_type in county_commission_elections_long.election_type.unique().tolist():
    com_election_type = county_commission_elections_long.query("election_type == @election_type")
    print(com_election_type.groupby("county_id_fips").election_id.nunique().value_counts())
    print()

Yes!

- select the closest upcoming election for each county/type
- pivot

In [ ]:
next_county_commission_elections_long = county_commission_elections_long.loc[county_commission_elections_long.groupby(["county_id_fips", "election_type"])["election_day"].idxmax()]

In [ ]:
next_county_commission_elections_wide = next_county_commission_elections_long.pivot(index=["county_id_fips","county_name"], columns=["election_type"])

next_county_commission_elections_wide.columns = next_county_commission_elections_wide.swaplevel(axis=1).columns
next_county_commission_elections_wide = next_county_commission_elections_wide.sort_index(axis=1, level='election_type')

next_county_commission_elections_wide.columns = next_county_commission_elections_wide.columns.map('_'.join)
next_county_commission_elections_wide.columns = ["next_" + col for col in next_county_commission_elections_wide.columns]

In [ ]:
next_county_commission_elections_wide = next_county_commission_elections_wide.reset_index().convert_dtypes()

assert next_county_commission_elections_wide.county_id_fips.is_unique, "county_id_fips is not unique!"

In [ ]:
next_county_commission_elections_wide.info()

In [ ]:
next_county_commission_elections_wide.next_run_off_election_id.isna().value_counts()

In [ ]:
e_type = "general"
invalid = ~(next_county_commission_elections_wide[f"next_{e_type}_total_n_seats"] >= next_county_commission_elections_wide[f"next_{e_type}_total_n_races"])
next_county_commission_elections_wide[invalid]

In [ ]:
"next_general_total_n_seats" == "next_general_total_n_of_races"

In [ ]:
data = {
    ('A', 'X'): [1, 2, 3],
    ('A', 'Y'): [4, 5, 6],
    ('B', 'X'): [7, 8, 9],
    ('B', 'Y'): [10, 11, 12],
}

df = pd.DataFrame(data)

# Concatenate the multi-index columns into a single index
df.columns = df.columns.map('_'.join)

print(df)

In [ ]:
next_county_commission_elections_wide

In [ ]:
next_county_commission_elections_wide.sort_index(axis=1, level='election_type').columns

In [ ]:
key=lambda x: x.map({'general': 1, 'primary': 2, 'runoff': 3}

In [ ]:
list(next_county_commission_elections_wide)

In [ ]:
next_county_commission_elections_wide.columns

In [ ]:
import pandas as pd

# Create a sample DataFrame with multi-index columns
data = {
    ('A', 'X'): [1, 2, 3],
    ('A', 'Y'): [4, 5, 6],
    ('B', 'X'): [7, 8, 9],
    ('B', 'Y'): [10, 11, 12],
}

df = pd.DataFrame(data)

df.columns = df.columns.map('_'.join)
df

In [ ]:
generals.groupby("county_id_fips").election_id.nunique().sort_values(ascending=False).head(10)

In [ ]:
generals.query("county_id_fips == '34027'")

In [ ]:
county_commission_elections_long.pivot(index="county_id_fips", columns=["election_type", "election_name"])